In [1]:
import jupyter_black

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchvision.io import read_image
from torchvision.io.image import ImageReadMode
from torch.utils.data import DataLoader

from PIL import Image

import sys

if not "/usr/src" in sys.path:
    sys.path.append("/usr/src/")

jupyter_black.load()

In [2]:
# Custom modules
from dataset import create_substrate_dataset, get_part_substrates_from_all
from unet import UNet
from diffusion import Diffusion

In [3]:
# Dirs
substrate_dir = "/usr/src/substrates/"
part_substrate_dir = "/usr/src/part_substrates/"
cell_dir = "/usr/src/separated_cells/"